## Preprocessing and Training Data Development - Vacancy Rates 
### Goal:  Create a cleaned development dataset you can use to complete the modeling step of your project


#### Steps: 
● 1. Create dummy or indicator features for categorical variables

● 2. Standardize the magnitude of numeric features using a scaler

● 3. Split into testing and training datasets

In [1]:
#imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime
from pandas_profiling import ProfileReport

In [2]:
#load data
path= '/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/interim'
os.chdir(path) 

In [6]:
DF = pd.read_csv('VacancyRate_Zipcode_AND_National_2011_2020.csv',  dtype={'Zipcode': object}, parse_dates=['Year'])
DF

,Zipcode,Vacancy_Rate%,MOE-VacancyRate%,Year
0,02333,3.024027,2.199925,2011-01-01
1,02338,3.116343,2.948791,2011-01-01
2,02339,4.464646,2.066438,2011-01-01
3,02341,3.586322,2.340722,2011-01-01
4,02343,3.732901,2.926524,2011-01-01
...,...,...,...,...
264965,NATNL,6.850000,0.000000,2016-01-01
264966,NATNL,7.175000,0.000000,2017-01-01
264967,NATNL,6.875000,0.000000,2018-01-01
264968,NATNL,6.750000,0.000000,2019-01-01


In [7]:
DF.dtypes

Zipcode                     object
Vacancy_Rate%              float64
MOE-VacancyRate%           float64
Year                datetime64[ns]
dtype: object

In [8]:
#Create a new dataframe, setting the index to 'Year'
df = DF.set_index('Year')
#Save the DATE labels 
df_index = df.index
#Save the column names
df_columns = df.columns
df.head()

,Zipcode,Vacancy_Rate%,MOE-VacancyRate%
Year,,,
2011-01-01,02333,3.024027,2.199925
2011-01-01,02338,3.116343,2.948791
2011-01-01,02339,4.464646,2.066438
2011-01-01,02341,3.586322,2.340722
2011-01-01,02343,3.732901,2.926524


In [10]:
#check for NaNs
df.isna().sum()

Zipcode             0
Vacancy_Rate%       0
MOE-VacancyRate%    0
dtype: int64

In [26]:
#check unique values for each column
df['MOE-VacancyRate%'].value_counts()/len(df)*100

0.000000      5.630071
100.000000    0.207571
20.000000     0.046043
33.333333     0.040759
25.000000     0.037363
                ...   
8.671855      0.000377
17.389154     0.000377
2.754040      0.000377
26.238062     0.000377
5.466803      0.000377
Name: MOE-VacancyRate%, Length: 234862, dtype: float64

In [35]:
#change National ('NATNL') zipcode to '99999' for later modeling
df.Zipcode.replace('NATNL', '99999', inplace=True)
df.Zipcode = df.Zipcode.astype('int')
df.dtypes

Zipcode               int64
Vacancy_Rate%       float64
MOE-VacancyRate%    float64
dtype: object

In [36]:
#check partition sizes with a 80/20 train/test split for all DataFrames
print('train size:', len(df) * .80, 'test size:', len(df) * .20)

train size: 211976.0 test size: 52994.0


### 1. Split into testing and training datasets
Hint: don’t forget your sklearn functions here, like train_test_split().

In [47]:
#separate 2020 data from data set for later use and prediction
df_2019AND20 = df[df.index > '2018']
df = df[df.index < '2019']

In [48]:
#define variable X, y
X = df.drop('Vacancy_Rate%', axis=1)
y = df['Vacancy_Rate%']

In [51]:
#train test split each of the 4 DFs
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

### Establish Baseline Measurement Comparisons
Using a Dummy Regressor see what R2, MSE, and MAE would be if the mean of the DataFrames were used

In [52]:
#initial not even a model
train_mean = y_train.mean()

print(train_mean)

17.688364513059945


In [53]:
#Fit the dummy regressor on the training data
dumb_reg = DummyRegressor(strategy='mean')
dumb_reg.fit(X_train, y_train)
#create dummy regressor predictions 
y_tr_pred = dumb_reg.predict(X_train)
#Make prediction with the single value of the (training) mean.
y_te_pred = train_mean * np.ones(len(y_test))
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

(0.0, -1.7221596428695918e-05)

In [54]:
#establish baseline for mean absolute error and mean square error 
print('MAEs:', mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred))
print('MSEs:', mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred))

MAEs: 11.766127007670223 11.703398452536309
MSEs: 270.86710986879007 267.5387213119894


###  2. Create dummy or indicator features for categorical variables
Hint: you’ll need to think about your old favorite pandas functions here like
get_dummies() . Consult this guide for help.
<https://towardsdatascience.com/the-dummys-guide-to-creating-dummy-variables-f21faddb1d40>

In [31]:
#step not needed as the zipcode variable was changed to integer above

### 3. Standardize the magnitude of numeric features using a scaler
Hint: you might need to employ Python code like this:

In [32]:
'''
# Making a Scaler object
scaler = preprocessing.StandardScaler()
# Fitting data to the scaler object
scaled_df = scaler.fit_transform(df)
scaled_df = pd.DataFrame(scaled_df, columns=names)
'''

'\n# Making a Scaler object\nscaler = preprocessing.StandardScaler()\n# Fitting data to the scaler object\nscaled_df = scaler.fit_transform(df)\nscaled_df = pd.DataFrame(scaled_df, columns=names)\n'

In [33]:
#don't need because everythin is % (0-100)
'''
scaler = StandardScaler()
#fit the scaler on the training set
scaler.fit(X_train)
#apply the scaling to both the train and test split
X_tr_scaled = scaler.transform(X_train)
X_te_scaled = scaler.transform(X_test)
'''

#### Initial Model: Train the model on the train split

In [56]:
lm = LinearRegression().fit(X_train, y_train)

In [57]:
#Make predictions using the model on both train and test splits
y_tr_pred = lm.predict(X_train)
y_te_pred = lm.predict(X_test)

In [59]:
#Assess model performance
# r^2 - train, test
r2 = r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)
print('r2:', r2)

r2: (0.48181793766528913, 0.48624846101375807)


**This is markedly better performance than when using Dummy variable/mean for R^2 (see earlier):**

Dummy R2 = (0.0, -1.7221596428695918e-05)

**Note: It does seem as is this model may be overfitting, as the y-test performs slightly better than the y-train**


In [60]:
#MAE - train, test
mae = mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)
print('mae:', mae)

mae: (8.013226732182535, 7.935339791599616)


In [61]:
# MSE - train, test
mse = mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred)
print('mse:', mse)

mse: (140.35847761045238, 137.44606277182206)


**This is markedly better performance than when using Dummy variable/mean for R^2 (see earlier):**

Dummy -

MAEs: 11.766127007670223 11.703398452536309

MSEs: 270.86710986879007 267.5387213119894

**MSE still very high (possibly due to this being a large data set**

## Save processed data

In [42]:
#save vacancy rate data for modeling - remember to use random state=42!
df.to_csv(r'/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/processed/VacancyRate_Zipcode_AND_National_2011_2018', index=False)
df_2019AND20.to_csv(r'/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/processed/VacancyRate_Zipcode_National_2019_2020', index=False)

In [62]:
#save the scaled training and test splits

#X_tr_scaled.to_csv(r'/Users/josephfrasca/Coding_Stuff/Springboard/Capstone_2/data/processed/X_tr_scaled', index=False)
#X_te_scaled

### Summary
This summary should provide a quick overview for someone wanting to know quickly why the given model was chosen for the next part of the business problem to help guide important business decisions.

In [ ]:
#complete summary

### Reflection: 
**Review the following questions and apply them to your dataset**:

● Does my data set have any categorical data, such as Gender or day of the week?

● Do my features have data values that range from 0 - 100 or 0-1 or both and more

In [ ]:
#try to add in categorical varirables: city, metro, sizerank etc.
    #need to create pd.get_dummies to handle this
#add in other FRED/ACS data points from 2011-2018 (both zipcode and national)
    #train model on scaled data!!! (after adding in other variables)